# Prepare Environment (default operations)

In [7]:
import os, numpy as np, tensorflow as tf, json, matplotlib.pyplot as plt
from tensorflow import keras
from data_loader.hgf_export import HGFresource
from model_accelerator.custom_loop import CustomModel
from dotenv import load_dotenv

In [2]:
load_dotenv()
HGF_TOKEN = os.environ['HUGGINGFACE_TOKEN']
HGF_DATA_REPO = os.environ['HUGGINGFACE_DATASET_V2_REPO']
HGF_BASELINE_MODEL_REPO = os.environ['HUGGINGFACE_BASELINE_CNN_REPO']
HGF_TOPIC_MODEL_TOP2VEC_REPO = os.environ['HUGGINGFACE_TOPIC_MODEL_TOP2VEC_REPO']
HGF_TOPIC_MODEL_TOP2VEC_FILE = os.environ['HUGGINGFACE_TOPIC_MODEL_TOP2VEC_FILE']

In [3]:
hgf = HGFresource(token=HGF_TOKEN)

In [6]:
topic_model = hgf.load_model(repo=HGF_TOPIC_MODEL_TOP2VEC_REPO, filename=HGF_TOPIC_MODEL_TOP2VEC_FILE)
N_CLASSES = topic_model.get_num_topics()

In [7]:
train_data, test_data = hgf.load_data_tfds(repo=HGF_DATA_REPO, n_classes=N_CLASSES, batch_size=32)

In [11]:
# learning rate may require fine-tuning but use the optimizer specified below
OPTIMIZER = tf.keras.optimizers.legacy.Adam(learning_rate=0.001) # use this if you have M1/M2 Mac, otherwise use the next line
# OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=0.001)
LOSS = 'categorical_crossentropy'
METRICS = [tf.keras.metrics.F1Score('weighted')]
EPOCHS = 20
INPUT_SHAPE = (256, 219, 3)

In [9]:
def history_plot(history):
    train_f1_scores = history.history['f1_score']
    val_f1_scores = history.history['val_f1_score']

    epochs = range(1, len(train_f1_scores) + 1)
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, train_f1_scores, 'bo-', label='Training F1 Score')
    plt.plot(epochs, val_f1_scores, 'ro-', label='Validation F1 Score')
    plt.title('Training and Validation F1 Score')
    plt.xlabel('Epochs')
    plt.ylabel('F1 Score')
    plt.legend()
    plt.grid(True)
    plt.show()

# Your Code Starts Here

In [10]:
inputs = keras.Input(shape=(256, 219, 3))
conv1 = keras.layers.Conv2D(16, (3, 3), activation='relu')(inputs)
maxpool1 = keras.layers.MaxPool2D(pool_size=(2, 2), padding='same')(conv1)
conv2 = keras.layers.Conv2D(32, (3, 3), activation='relu')(maxpool1)
maxpool2 = keras.layers.MaxPool2D(pool_size=(2, 2))(conv2)
conv3 = keras.layers.Conv2D(64, (3, 3), activation='relu')(maxpool2)
maxpool3 = keras.layers.MaxPool2D(pool_size=(2, 2))(conv3)
flat = keras.layers.Flatten()(maxpool3)
dense = keras.layers.Dense(128, activation='relu')(flat)
outputs = keras.layers.Dense(N_CLASSES, activation='softmax')(dense)
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

In [11]:
model.compile(
    OPTIMIZER,
    LOSS,
    METRICS
)

In [ ]:
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=EPOCHS
)

In [ ]:
history_plot(history)

# Commit Your Results to Hugging Face Once Done

In [48]:
# NOTE: when naming your weights file
# indicate the type of model
# e.g.:
## 'tuned_optimizer_model_weights.h5'
## 'data_augmentation_model_weights.h5'
WEIGHTS_TO_COMMIT = 'baseline_model_weights.h5'
# save model weights
model.save_weights(WEIGHTS_TO_COMMIT)

In [49]:
# save model config
CONFIG_TO_COMMIT  = 'model_config.json'
model_config = model.get_config()
model_config = json.dumps(model_config)
with open(CONFIG_TO_COMMIT, 'w') as f:
    f.write(model_config)

In [50]:
PERFORMANCE_TO_COMMIT = 'baseline_model_performance.json'
performance_dict = {
    'train': {
        'weighted_f1': history.history['f1_score'][-1]
    },
    'test': {
        'weighted_f1': history.history['val_f1_score'][-1]
    }
}
performance_dict = json.dumps(performance_dict)
with open(PERFORMANCE_TO_COMMIT, 'w') as f:
    f.write(performance_dict)

In [51]:
hgf.commit_to_hub(
    repo=HGF_BASELINE_MODEL_REPO,
    path_on_local=[WEIGHTS_TO_COMMIT, CONFIG_TO_COMMIT, PERFORMANCE_TO_COMMIT],
    path_in_repo=[WEIGHTS_TO_COMMIT, CONFIG_TO_COMMIT, PERFORMANCE_TO_COMMIT],
    # do not forget to change your commit message
    # to add even more clarity
    commit_message='upload baseline model weights, config, and scores'
)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/fedorturchenko/.cache/huggingface/token
Login successful
Successfully logged out.


In [52]:
# remove files from local = clean up
for path in [WEIGHTS_TO_COMMIT, CONFIG_TO_COMMIT, PERFORMANCE_TO_COMMIT]:
    os.remove(path)

# Load Model from Hugging Face

In [4]:
HGF_BASELINE_MODEL_CONFIG = os.environ['HUGGINGFACE_BASELINE_CNN_CONFIG_FILE']
HGF_BASELINE_MODEL_WEIGHTS = os.environ['HUGGINGFACE_BASELINE_CNN_WEIGHTS_FILE']

In [8]:
# load config of baseline model which we need to optimize
model_config_file = hgf.load_file(repo=HGF_BASELINE_MODEL_REPO, filename=HGF_BASELINE_MODEL_CONFIG)
with open(model_config_file, 'r') as f:
    model_config = f.read()
model_config = json.loads(model_config)
# extract number of classes from the last layer
N_CLASSES = model_config['layers'][-1]['config']['units']

# alternatively, you can load the full baseline model with pre-defined weights
# you may want to do this for the sake of performance comparison
inputs, outputs, weights = hgf.load_model(
    repo=HGF_BASELINE_MODEL_REPO,
    filename={
        'model_weights': HGF_BASELINE_MODEL_WEIGHTS,
        'model_config': HGF_BASELINE_MODEL_CONFIG
    },
    model_type='func'
)

In [9]:
# initialize the baseline model architecture
model = CustomModel(inputs=inputs, outputs=outputs)
# load weights if needed
model.load_weights(weights)
# view the model architecture
model.summary()

Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 256, 219, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")
Model: "custom_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        multiple                     0         ['input_1[0][0]']             
                                                                                                  
 conv2d (Conv2D)             (None, 254, 217, 16)         448       ['input_1[1][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 127, 109, 16)         0         ['conv2d[0][0]']              
 D)            